In [2]:
import trainers.dpo as dpo
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer
)
import torch

model_id = "meta-llama/Llama-3.2-1B"


model = AutoModelForCausalLM.from_pretrained(
            model_id,device_map="cuda",load_in_8bit=True)

ref_model = AutoModelForCausalLM.from_pretrained(
            model_id,device_map="cuda",load_in_8bit=True)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


In [3]:
from tokenizer.tokenization import tokenize_dataset,collate_fn


ds = load_dataset("HumanLLMs/Human-Like-DPO-Dataset")
# 訓練データのサブセットを選択
dataset = ds['train'].select(range(200))

tokenized_dataset = dataset.map(
    lambda batch: tokenize_dataset(tokenizer, batch),
)



In [4]:
from torch.utils.data import DataLoader, Dataset

class TokenizedDataset(Dataset):
    def __init__(self, tokenized_data):
        """
        トークナイズされたデータを直接受け取る
        """
        self.data = tokenized_data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [5]:
# トークナイズされたデータセットをリスト形式に変換
tokenized_data = list(tokenized_dataset)

# カスタムデータセットのインスタンスを作成
custom_dataset = TokenizedDataset(tokenized_data)
# データローダーの作成
batch_size = 4
dataloader = DataLoader(
    custom_dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn  # 修正済みの collate_fn を指定
)

In [6]:
dataloader

In [ ]:


beta = 0.5

dpo_model = dpo.DPO(model,ref_model,beta)


In [ ]:

# トレーニング実行
dpo_model.train(dataloader, num_epochs=1)

In [ ]:
save_directory = "./saved_model"

dpo_model.model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)


('./saved_model\\tokenizer_config.json',
 './saved_model\\special_tokens_map.json',
 './saved_model\\tokenizer.json')

In [ ]:
from transformers import AutoTokenizer,set_seed


tokenizer = AutoTokenizer.from_pretrained(model_id)

input_ids = tokenizer.encode(ds['train'][0]["prompt"]+"\n", return_tensors="pt").to("cuda")
set_seed(54)
output =dpo_model.model.generate(input_ids,do_sample=True,top_p=0.9,max_new_tokens=24)



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [ ]:
print(tokenizer.decode(output[0], skip_special_tokens=True))


Oh, I just saw the best meme - have you seen it?
I don't know who posted it, but it's so good that I just had to share it. It's a


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

input_ids = tokenizer.encode(ds['train'][0]["prompt"]+"\n", return_tensors="pt").to(dpo_model.ref_model.device)

set_seed(54)

output =dpo_model.ref_model.generate(input_ids,do_sample=True,top_p=0.9,max_new_tokens=24)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [ ]:
print(tokenizer.decode(output[0], skip_special_tokens=True))


Oh, I just saw the best meme - have you seen it?
I don't know who posted it, but it's so good that I just had to share it. It's a


In [ ]:
print(torch.cuda.is_available())


True
